# Detecting pneumonia

### First some imports

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
from glob import glob

### Specific tensorflow + keras imports

In [110]:
#import tensorflow as tf
import tensorflow as tf

from keras.utils import to_categorical
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model 


from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile



## Data

In [80]:
batch_size = 16
test_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)


validation_generator = test_datagen.flow_from_directory(
        './images/val',
        target_size=(299, 299),
        batch_size=batch_size)
train_generator = train_datagen.flow_from_directory(
        './images/train',
        target_size=(299, 299),
        batch_size=batch_size)

Found 519 images belonging to 3 classes.
Found 5098 images belonging to 3 classes.


In [64]:

def get_image_files(image_dir):
  fs = glob("{}/*.jpeg".format(image_dir))
  fs = [os.path.basename(filename) for filename in fs]
  return sorted(fs)







## Define model

In [114]:
model = applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(299, 299, 3), pooling='avg')

In [118]:
model.summary()

for layer in model.layers:
    layer.trainable = False


AttributeError: 'NoneType' object has no attribute 'summary'

In [117]:


x = Dense(768, activation='relu')(model.output)
x = Dense(384, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model_final = Model(inputs = model.input, outputs = predictions)

model = model_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


AttributeError: 'NoneType' object has no attribute 'output'

In [ ]:
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

## Compile model

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Fit model

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)